In [1]:
import torch

In [15]:
torch.manual_seed(0) # 固定随机数种子

word_emb=torch.rand(5,3) # 5个词,每个词的emd宽3
w_q=torch.rand(3,3) 
w_k=torch.rand(3,3)
w_v=torch.rand(3,3)
q_1=torch.matmul(word_emb,w_q)
k_1=torch.matmul(word_emb,w_k)
print('q_1',q_1)
print('k_1',k_1)

print('k_1_transpose',k_1.transpose(1,0))
a=q_1@(k_1.transpose(1,0))
a_=torch.nn.functional.softmax(a,-1)
print('attension score', a_)
v_1=torch.matmul(word_emb,w_v)
print('v1', v_1)
b=v_1.T@a_.T
print('b1~-b4',b)
print('b1',b[:,0])

q_1 tensor([[0.4620, 0.5982, 0.9980],
        [0.6983, 0.4307, 0.8695],
        [0.8155, 0.7759, 1.4014],
        [0.7517, 0.6991, 1.0948],
        [0.3077, 0.1799, 0.3899]])
k_1 tensor([[0.2629, 0.4991, 0.7752],
        [0.2599, 0.7209, 0.3521],
        [0.3770, 0.8617, 0.8818],
        [0.4004, 0.7886, 0.8034],
        [0.1051, 0.3175, 0.1360]])
k_1_transpose tensor([[0.2629, 0.2599, 0.3770, 0.4004, 0.1051],
        [0.4991, 0.7209, 0.8617, 0.7886, 0.3175],
        [0.7752, 0.3521, 0.8818, 0.8034, 0.1360]])
attension score tensor([[0.2021, 0.1511, 0.2944, 0.2634, 0.0891],
        [0.2040, 0.1550, 0.2833, 0.2607, 0.0969],
        [0.1956, 0.1281, 0.3303, 0.2850, 0.0610],
        [0.1961, 0.1438, 0.3094, 0.2746, 0.0762],
        [0.2056, 0.1813, 0.2369, 0.2284, 0.1477]])
v1 tensor([[0.3595, 0.8356, 0.5928],
        [0.5703, 0.6394, 0.3810],
        [0.6570, 1.1472, 0.7748],
        [0.5414, 0.6311, 0.3702],
        [0.2593, 0.3150, 0.1944]])
b1~-b4 tensor([[0.5179, 0.5142, 0.5305, 0.52